In [ ]:
!git clone https://github.com/yzhou359/MakeItTalk.git

Cloning into 'MakeItTalk'...
remote: Enumerating objects: 650, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 650 (delta 57), reused 50 (delta 50), pack-reused 558
Receiving objects: 100% (650/650), 26.45 MiB | 17.82 MiB/s, done.
Resolving deltas: 100% (154/154), done.


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached ffmpeg_python-0.2.0-py3-none-any.whl (25 kB)
  Using cached face_alignment-1.4.0-py2.py3-none-any.whl (29 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
  Using cached pynormalize-0.1.4-py2.py3-none-any.whl (6.5 kB)
  Using cached pysptk-0.2.0.tar.gz (276 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.7/218.7 kB 19.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 8.7 MB/s eta 0:00:00
  Preparing metadat

In [ ]:
!pip install tensorboardX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.6 MB/s eta 0:00:00


In [ ]:
%cd MakeItTalk

[Errno 2] No such file or directory: 'MakeItTalk'
/content/MakeItTalk


In [ ]:
import sys
sys.path.append("thirdparty/AdaptiveWingLoss")
import os, glob
import numpy as np
import cv2
import argparse
from src.approaches.train_image_translation import Image_translation_block
import torch
import pickle
import face_alignment
from src.autovc.AutoVC_mel_Convertor_retrain_version import AutoVC_mel_Convertor
import shutil
import time
import util.utils as util
from scipy.signal import savgol_filter
from src.approaches.train_audio2landmark import Audio2landmark_model

In [ ]:
!mkdir examples/dump
!mkdir examples/ckpt
!pip install gdown
!gdown -O examples/ckpt/ckpt_autovc.pth https://drive.google.com/uc?id=1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x
!gdown -O examples/ckpt/ckpt_content_branch.pth https://drive.google.com/uc?id=1r3bfEvTVl6pCNw5xwUhEglwDHjWtAqQp
!gdown -O examples/ckpt/ckpt_speaker_branch.pth https://drive.google.com/uc?id=1rV0jkyDqPW-aDJcj7xSO6Zt1zSXqn1mu
!gdown -O examples/ckpt/ckpt_116_i2i_comb.pth https://drive.google.com/uc?id=1i2LJXKp-yWKIEEgJ7C6cE3_2NirfY_0a
!gdown -O examples/dump/emb.pickle https://drive.google.com/uc?id=18-0CYl5E6ungS3H4rRSHjfYvvm-WwjTI

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Downloading...
From: https://drive.google.com/uc?id=1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x
To: /content/MakeItTalk/examples/ckpt/ckpt_autovc.pth
100% 172M/172M [00:04<00:00, 36.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1r3bfEvTVl6pCNw5xwUhEglwDHjWtAqQp
To: /content/MakeItTalk/examples/ckpt/ckpt_content_branch.pth
100% 7.88M/7.88M [00:00<00:00, 20.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1rV0jkyDqPW-aDJcj7xSO6Zt1zSXqn1mu
To: /content/MakeItTalk/examples/ckpt/ckpt_speaker_branch.pth
100% 15.4M/15.4M [00:00<00:00, 22.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1i2LJXKp-yWKIEEgJ7C6cE3_2NirfY_0a
To: /content/MakeItTalk/examples/ckpt/ckpt_116_i2i_comb.pth
100% 839M/839M [00:15<00:00, 55.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=18-0CYl5E6ungS3H4rRSHjfYvvm-WwjTI
To: /content/MakeItTalk/examples/dump/emb.pickle
100% 30.9M/30.9M [00:01<

In [ ]:
default_head_name = 'paint_boy'
ADD_NAIVE_EYE = True
CLOSE_INPUT_FACE_MOUTH = False
AMP_LIP_SHAPE_X = 2.
AMP_LIP_SHAPE_Y = 2.
AMP_HEAD_POSE_MOTION = 0.7

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--jpg', type=str, default='{}.jpg'.format(default_head_name))
parser.add_argument('--close_input_face_mouth', default=CLOSE_INPUT_FACE_MOUTH, action='store_true')

parser.add_argument('--load_AUTOVC_name', type=str, default='examples/ckpt/ckpt_autovc.pth')
parser.add_argument('--load_a2l_G_name', type=str, default='examples/ckpt/ckpt_speaker_branch.pth')
parser.add_argument('--load_a2l_C_name', type=str, default='examples/ckpt/ckpt_content_branch.pth') #ckpt_audio2landmark_c.pth')
parser.add_argument('--load_G_name', type=str, default='examples/ckpt/ckpt_116_i2i_comb.pth') #ckpt_image2image.pth') #ckpt_i2i_finetune_150.pth') #c

parser.add_argument('--amp_lip_x', type=float, default=AMP_LIP_SHAPE_X)
parser.add_argument('--amp_lip_y', type=float, default=AMP_LIP_SHAPE_Y)
parser.add_argument('--amp_pos', type=float, default=AMP_HEAD_POSE_MOTION)
parser.add_argument('--reuse_train_emb_list', type=str, nargs='+', default=[]) #  ['iWeklsXc0H8']) #['45hn7-LXDX8']) #['E_kmpT-EfOg']) #'iWeklsXc0H8', '29k8RtSUjE0', '45hn7-LXDX8',
parser.add_argument('--add_audio_in', default=False, action='store_true')
parser.add_argument('--comb_fan_awing', default=False, action='store_true')
parser.add_argument('--output_folder', type=str, default='examples')

parser.add_argument('--test_end2end', default=True, action='store_true')
parser.add_argument('--dump_dir', type=str, default='', help='')
parser.add_argument('--pos_dim', default=7, type=int)
parser.add_argument('--use_prior_net', default=True, action='store_true')
parser.add_argument('--transformer_d_model', default=32, type=int)
parser.add_argument('--transformer_N', default=2, type=int)
parser.add_argument('--transformer_heads', default=2, type=int)
parser.add_argument('--spk_emb_enc_size', default=16, type=int)
parser.add_argument('--init_content_encoder', type=str, default='')
parser.add_argument('--lr', type=float, default=1e-3, help='learning rate')
parser.add_argument('--reg_lr', type=float, default=1e-6, help='weight decay')
parser.add_argument('--write', default=False, action='store_true')
parser.add_argument('--segment_batch_size', type=int, default=1, help='batch size')
parser.add_argument('--emb_coef', default=3.0, type=float)
parser.add_argument('--lambda_laplacian_smooth_loss', default=1.0, type=float)
parser.add_argument('--use_11spk_only', default=False, action='store_true')
parser.add_argument('-f')

opt_parser = parser.parse_args()

In [ ]:
img =cv2.imread("/content/MakeItTalk/examples/anne.jpg")
predictor = face_alignment.FaceAlignment(face_alignment.LandmarksType._3D, device='cpu', flip_input=True)
shapes = predictor.get_landmarks(img)
if (not shapes or len(shapes) != 1):
    print('Cannot detect face landmarks. Exit.')
    exit(-1)
shape_3d = shapes[0]

if(opt_parser.close_input_face_mouth):
    util.close_input_face_mouth(shape_3d)

AttributeError: ignored

In [ ]:
import tkinter as tk
import os
import torch
import torchaudio
from transformers import AutoModelForCausalLM, AutoTokenizer
import vlc

ModuleNotFoundError: ignored

In [ ]:
os.system("xcopy /y generated.wav .\MakeItTalk\examples")
os.system("cd MakeItTalk & python generate.py")

# if os.path.exists('generated.wav'):
p = vlc.MediaPlayer("/content/file_example_WAV_1MG.wav")
p.play()



In [ ]:
!git clone https://github.com/yzhou359/MakeItTalk.git

Cloning into 'MakeItTalk'...
remote: Enumerating objects: 650, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 650 (delta 50), reused 44 (delta 44), pack-reused 570
Receiving objects: 100% (650/650), 26.46 MiB | 20.32 MiB/s, done.
Resolving deltas: 100% (153/153), done.


In [ ]:
# os.system("xcopy /y /content/file_example_WAV_1MG.wav .\MakeItTalk\examples")
os.system("cd MakeItTalk & python main_end2end.py")

512

In [ ]:
%cd ..
# %cd examples

/content/MakeItTalk


In [ ]:
!wget

In [ ]:
!python main_end2end.py --jpg anne2.jpg

/content/MakeItTalk/main_end2end.py:99: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  ains = [item for item in ains if item is not 'tmp.wav']
Loaded the voice encoder model on cuda in 0.02 seconds.
Processing audio file M6_04_16k.wav
0 out of 0 are in this portion
Loaded the voice encoder model on cuda in 0.01 seconds.
source shape: torch.Size([1, 320, 128]) torch.Size([1, 256]) torch.Size([1, 256]) torch.Size([1, 320, 257])
Traceback (most recent call last):
  File "/content/MakeItTalk/main_end2end.py", line 113, in <module>
    au_data_i = c.convert_single_wav_to_autovc_input(audio_filename=os.path.join('examples', ain),
  File "/content/MakeItTalk/src/autovc/AutoVC_mel_Convertor_retrain_version.py", line 260, in convert_single_wav_to_autovc_input
    x_identic, x_identic_psnt_i, code_real = G(x_real, emb_org, f0_org, emb_trg, f0_org)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **k

In [ ]:
!wget --no-check-certificate "https://docs.google.com/uc?export=download&id=1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x"

--2023-05-30 14:56:15--  https://docs.google.com/uc?export=download&id=1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x
Resolving docs.google.com (docs.google.com)... 173.194.79.101, 173.194.79.102, 173.194.79.139, ...
Connecting to docs.google.com (docs.google.com)|173.194.79.101|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘uc?export=download&id=1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x’

uc?export=download&     [ <=>                ]   2.22K  --.-KB/s    in 0s      

2023-05-30 14:56:16 (41.4 MB/s) - ‘uc?export=download&id=1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x’ saved [2277]



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!dir

CODE_OF_CONDUCT.md  main_end2end_cartoon.py	     quick_demo_tdlr.ipynb
CONTRIBUTING.md     main_end2end.py		     README.md
doc		    main_gen_new_puppet.py	     requirements.txt
examples	    main_train_content.py	     src
examples_cartoon    main_train_image_translation.py  thirdparty
facewarp	    main_train_speaker_aware.py      util
gypsum_history.sh   PULL_REQUEST_TEMPLATE.md
LICENSE.md	    quick_demo.ipynb


In [ ]:
%cd ..

/content


In [ ]:
!wget -O ebook.pdf --no-check-certificate  https://drive.google.com/file/d/1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x/view?usp=sharing

--2023-05-30 08:52:17--  https://drive.google.com/file/d/1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x/view?usp=sharing
Resolving drive.google.com (drive.google.com)... 142.250.145.139, 142.250.145.113, 142.250.145.102, ...
Connecting to drive.google.com (drive.google.com)|142.250.145.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘ebook.pdf’

ebook.pdf               [ <=>                ]  73.56K  --.-KB/s    in 0.008s  

2023-05-30 08:52:18 (8.82 MB/s) - ‘ebook.pdf’ saved [75324]



In [ ]:
!wget  "https://drive.google.com/file/d/1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x/view?usp=sharing"

--2023-05-30 09:08:06--  https://drive.google.com/file/d/1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x/view?usp=sharing
Resolving drive.google.com (drive.google.com)... 108.177.127.139, 108.177.127.102, 108.177.127.138, ...
Connecting to drive.google.com (drive.google.com)|108.177.127.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘view?usp=sharing’

view?usp=sharing        [ <=>                ]  73.56K  --.-KB/s    in 0.01s   

2023-05-30 09:08:07 (6.88 MB/s) - ‘view?usp=sharing’ saved [75325]



In [ ]:
!wget https://drive.google.com/open?id=1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x&authuser=0

--2023-05-30 09:02:51--  https://drive.google.com/open?id=1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x
Resolving drive.google.com (drive.google.com)... 108.177.119.101, 108.177.119.113, 108.177.119.100, ...
Connecting to drive.google.com (drive.google.com)|108.177.119.101|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://drive.google.com/file/d/1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x/view?usp=drive_open [following]
--2023-05-30 09:02:52--  https://drive.google.com/file/d/1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x/view?usp=drive_open
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘open?id=1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x’

open?id=1ZiwPp_h62L     [ <=>                ]  73.60K  --.-KB/s    in 0.01s   

2023-05-30 09:02:52 (7.15 MB/s) - ‘open?id=1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x’ saved [75362]



In [ ]:
!wget https://drive.google.com/u/0/uc?id=1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x&export=download&confirm=t&uuid=3e94d115-e395-4a76-aa3d-fc1b9bc1f42b&at=AKKF8vyemhf-EYtjqsJwIJHlxmsp:1685436852579

--2023-05-30 09:04:30--  https://drive.google.com/u/0/uc?id=1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x
Resolving drive.google.com (drive.google.com)... 108.177.127.138, 108.177.127.100, 108.177.127.139, ...
Connecting to drive.google.com (drive.google.com)|108.177.127.138|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://drive.google.com/uc?id=1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x [following]
--2023-05-30 09:04:31--  https://drive.google.com/uc?id=1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘uc?id=1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x’

uc?id=1ZiwPp_h62Ltj     [ <=>                ]   2.21K  --.-KB/s    in 0s      

2023-05-30 09:04:31 (25.3 MB/s) - ‘uc?id=1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x’ saved [2258]



In [ ]:
!wget 1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x.zip

--2023-05-30 09:10:18--  http://1ziwpp_h62ltju0dwpelluoodkpr85k7x.zip/
Resolving 1ziwpp_h62ltju0dwpelluoodkpr85k7x.zip (1ziwpp_h62ltju0dwpelluoodkpr85k7x.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘1ziwpp_h62ltju0dwpelluoodkpr85k7x.zip’


In [ ]:
!wget -c "https://drive.google.com/uc?export=download&id=1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x"

--2023-05-30 15:08:18--  https://drive.google.com/uc?export=download&id=1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x
Resolving drive.google.com (drive.google.com)... 142.250.153.138, 142.250.153.139, 142.250.153.102, ...
Connecting to drive.google.com (drive.google.com)|142.250.153.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘uc?export=download&id=1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x’

uc?export=download&     [ <=>                ]   2.22K  --.-KB/s    in 0s      

2023-05-30 15:08:18 (90.2 MB/s) - ‘uc?export=download&id=1ZiwPp_h62LtjU0DwpelLUoodKPR85K7x’ saved [2278]



In [ ]:
!unzip /content/drive/MyDrive/MakeItTalk.zip

In [ ]:
!dir

drive  MakeItTalk  sample_data


In [ ]:
%cd MakeItTalk

/content/MakeItTalk
